In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv',index_col='id')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv',index_col='id')
submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv',index_col='id')

In [ ]:
# Let's assume all features are categorical. But some values exist only in train or test
# Clean it manually

test.loc[test['feature_3']==25,'feature_3']=26
test.loc[test['feature_4']==36,'feature_4']=37
test.loc[test['feature_21']==31,'feature_21']=36
test.loc[test['feature_25']==24,'feature_25']=23
test.loc[test['feature_34']==26,'feature_34']=25
test.loc[test['feature_49']==21,'feature_49']=20

print(len(train))
train = train[train['feature_5']!=10]

train = train[train['feature_6']!=26]
train = train[train['feature_6']!=27]

train = train[train['feature_7']!=30]
train = train[train['feature_7']!=31]

train = train[train['feature_9']!=17]

train = train[train['feature_10']!=16]

train = train[train['feature_11']!=12]

train = train[train['feature_15']!=20]

train = train[train['feature_16']!=18]

train = train[train['feature_23']!=18]
train = train[train['feature_23']!=19]

train = train[train['feature_27']!=29]

train = train[train['feature_28']!=23]

train = train[train['feature_29']!=13]

train = train[train['feature_33']!=24]

train = train[train['feature_32']!=26]
train = train[train['feature_32']!=27]

train = train[train['feature_35']!=43]
train = train[train['feature_35']!=-2]
train = train[train['feature_35']!=38]
train = train[train['feature_35']!=39]


train = train[train['feature_38']!=65]
train = train[train['feature_38']!=55]
train = train[train['feature_38']!=-8]
train = train[train['feature_38']!=-3]
train = train[train['feature_38']!=-2]
train = train[train['feature_38']!=63]

train = train[train['feature_39']!=65]
train = train[train['feature_39']!=66]
train = train[train['feature_39']!=-5]
train = train[train['feature_39']!=-3]
train = train[train['feature_39']!=-2]
train = train[train['feature_39']!=63]

train = train[train['feature_42']!=37]
train = train[train['feature_42']!=-2]
train = train[train['feature_42']!=-1]

train = train[train['feature_43']!=33]
train = train[train['feature_43']!=31]

print(len(train))

In [ ]:
columns = train.columns[:-1]
cat_features = ['feature_0',
                'feature_2',
                'feature_3',
                'feature_4',
                'feature_5',
                'feature_6',
                'feature_7',
                'feature_9',
                'feature_10',
                'feature_11',
                'feature_12',
                'feature_13',
                'feature_15',
                'feature_16',
                'feature_17',
                'feature_18',
                'feature_20',
                'feature_21',
                'feature_22',
                'feature_23',
                'feature_25',
                'feature_26',
                'feature_27',
                'feature_28',
                'feature_29',
                'feature_32',
                'feature_33',
                'feature_34',
                'feature_35',
                'feature_36',
                'feature_38',
                'feature_39',
                'feature_42',
                'feature_43',
                'feature_44',
                'feature_46',
                'feature_49']

In [ ]:
def train_models(df, columns, cat_features, random_state, n_splits):

    models = []
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True,
                            random_state=random_state)  # create folds
    X_train = df[columns]
    y_train = df['target']
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X_train,  y_train)):
        print('Fold {}/{}'.format(n_fold, n_splits))
        train_X, train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_X, valid_y = X_train.iloc[valid_idx], y_train.iloc[valid_idx]
        dataset = Pool(train_X, train_y, cat_features)
        evalset = Pool(valid_X, valid_y, cat_features)
        model = CatBoostClassifier(
            task_type="GPU",
            depth=4,
            max_ctr_complexity=15,
            # border_count=1024,
            iterations=50000,
            od_wait=1000, od_type='Iter',
            # l2_leaf_reg=0.01,
            learning_rate=0.01,
            min_data_in_leaf=1,
            use_best_model=True,
            loss_function='MultiClass'

        )
        model.fit(dataset, plot=False, verbose=500, eval_set=evalset)        
        _record = {
            'model': model,
        }
        models.append(_record)

    return models

In [ ]:
models = train_models(df=train, columns=columns, cat_features=cat_features, random_state=2021, n_splits=9,)

In [ ]:
c1_columns = []
c2_columns = []
c3_columns = []
c4_columns = []

for idx,m in enumerate(models):
    print(idx)
    _proba = m['model'].predict_proba(test[columns])
    
    c1_name = 'm{}_c1'.format(idx)
    c1_columns.append(c1_name)
    test[c1_name] = _proba[:,0]
    
    c2_name = 'm{}_c2'.format(idx)
    c2_columns.append(c2_name)
    test[c2_name] = _proba[:,1]
    
    c3_name = 'm{}_c3'.format(idx)
    c3_columns.append(c3_name)
    test[c3_name] = _proba[:,2]
    
    c4_name = 'm{}_c4'.format(idx)
    c4_columns.append(c4_name)
    test[c4_name] = _proba[:,3]
    
    

In [ ]:
test['Class_1'] = test[c1_columns].apply(np.mean,axis=1)
test['Class_2'] = test[c2_columns].apply(np.mean,axis=1)
test['Class_3'] = test[c3_columns].apply(np.mean,axis=1)
test['Class_4'] = test[c4_columns].apply(np.mean,axis=1)
test.to_csv('test3.csv')

In [ ]:
submission['Class_1'] = test['Class_1']
submission['Class_2'] = test['Class_2']
submission['Class_3'] = test['Class_3']
submission['Class_4'] = test['Class_4']
submission.to_csv('catboost-3.csv')